In [1]:
import numpy as np
import pandas as pd
import networkx as nx

import sys

sys.path.append("../")
import utils
from gcn_models import GIN, GATC, SAGE

from os import path
import torch
import os
import numpy as np

import torch
import torch.nn.functional as F

from torch_geometric.loader import DataLoader
from sklearn.metrics import f1_score, accuracy_score

In [3]:
early_stop_thresh = 25
best_macro_f1 = -1


train_dataset = utils.GraphDataset("../data/", "GunPoint", True, True, n_quantiles=40)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)

test_dataset = utils.GraphDataset("../data/", "GunPoint", False, True, n_quantiles=50)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))


model = GIN(1, [8, 16, 16, 16, 8], pool_method="avg")

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=5, mode="min", cooldown=2, factor=0.5, verbose=True
)

(50, 150)
(150, 150)


In [4]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        y_out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        loss = F.cross_entropy(y_out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader, return_loss=True):
    model.eval()
    y_pred = []
    y_true = []
    loss = 0
    for data in loader:
        y_out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        y_pred.append(y_out.argmax(dim=-1))
        y_true.append(data.y)
        if return_loss:
            loss += float(F.cross_entropy(y_out, data.y)) * data.num_graphs
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    if return_loss:
        return (
            f1_score(y_true=y_true, y_pred=y_pred, average="macro"),
            accuracy_score(y_true=y_true, y_pred=y_pred),
            loss / len(loader.dataset),
        )
    return f1_score(y_true=y_true, y_pred=y_pred, average="macro"), accuracy_score(
        y_true=y_true, y_pred=y_pred
    )

In [5]:
for epoch in range(1, 501):
    train()
    train_macro_f1, train_acc, train_loss = test(train_loader)
    test_macro_f1, test_acc, test_loss = test(test_loader, True)
    scheduler.step(train_loss)
    print(
        f"Epoch: {epoch:03d}, Train_Loss: {train_loss:02.4f},Test_Loss: {test_loss:02.4f},Train_f1: {train_macro_f1:01.4f},Test_f1: {test_macro_f1:01.4f},Train_acc: {train_acc:01.4f},Test_acc: {test_acc:01.4f}"
    )
    if test_macro_f1 > best_macro_f1:
        best_accuracy = test_macro_f1
        best_epoch = epoch
        torch.save(model.state_dict(), "../data/best_model.pth")
    elif epoch - best_epoch > early_stop_thresh:
        print(
            f"Early stopped training at epoch {epoch} best macro F1 {best_macro_f1} in {best_epoch}"
        )
        break

TypeError: linear(): argument 'input' (position 1) must be Tensor, not NoneType